## Создание нейросети для запоминания рисунка

In [ ]:
import numpy
import scipy.special
import imageio
import matplotlib.pyplot
%matplotlib inline
import random

### предустановки
- **loadWeights**  Вначале надо указать, нужно ли создавать новые файлы весов или использовать старые.
  Если нужно использовать уже сохранёные веса, то следует указать    
  ```loadWeights = True```   
  Если обучение на данной картинке только начинается, то указываем
  ```loadWeights = False```   
  при этом создадутся новые файлы с весами нейросети, которые будут обновлятся в процессе обучения.
- **imageFileNumber** Номер картинки. Картинки для обучения хранятся в папке **img/** и имеют имя: **номер**+**".png"**
- **learning_rate** Скорость обучения. Я использовал **0.1**
- **neuron_count** Количество нейронов в скрытых слоях **(2 слоя)**     
  Чем меньше, тем быстрее происходит обучение, но возможны проблемы на сложных рисунках.     
  У меня неплохо получалось при **30** нейронов в скрытых слоях     
  Здесь я оставил **150**, так как есть комплект обученных нейронок с этим количеством,    
  а для создания перехода, нужны нейронки, обученные на одной и той же конфигурации нейросети.     
  Все файлы весов хранятся в папке **"W-150/"** (по 3 файла на каждый рисунок из папки **"img/"**)   
  Если планируете использовать количество нейронов отличное от **150**, то нужно создать соответствующую папку    
  **"W-"** + **количество нейронов** в скрытых слоях
- **folder** Имя папки (+ слэш) где можно будет посмотреть результат обучения в виде последовательности картинок

In [ ]:
# ======== начало предустановок, задаваемых пользователем   ==========

loadWeights = False  # Если нужно создать новые
#loadWeights = True  # Если нужно использовать существующие веса

imageFileNumber= 1

learning_rate = 0.1
neuron_count = 150

folder = 'progres/'  # имя папки, где будут сохраняться скриншоты

# ======== конец предустановок, задаваемых пользователем   ==========

# генерируются имена для файлов с весами нейросети
WeightFile1 = 'W-' + str(neuron_count) + '/'+'w1_' + str(imageFileNumber) + '.npy'
WeightFile2 = 'W-' + str(neuron_count) + '/'+'w2_' + str(imageFileNumber) + '.npy'
WeightFile3 = 'W-' + str(neuron_count) + '/'+'w3_' + str(imageFileNumber) + '.npy'

# загружаем картинку
imageFile = 'img/' + str(imageFileNumber) + '.png'

im = imageio.imread(imageFile)
imout = imageio.imread(imageFile)
matplotlib.pyplot.imshow(im, interpolation='None')

# узнаём параметры картинки
height = im.shape[0]
width  = im.shape[1]

### нейросеть

In [ ]:
# neural network class definition
class neuralNetwork:
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes1, hiddennodes2, outputnodes, learningrate, loadWeights = False):
        # set number of nodes in each input, hidden, output layer
        self.inpNodes = inputnodes
        self.hd1Nodes = hiddennodes1
        self.hd2Nodes = hiddennodes2
        self.outNodes = outputnodes
        
        if(loadWeights):
            self.wih1 = numpy.load(WeightFile1)
            self.wh1h2= numpy.load(WeightFile2)
            self.wh2o = numpy.load(WeightFile3)          
        else:
            self.wih1  = numpy.random.rand(self.hd1Nodes, self.inpNodes)/10-0.05
            self.wh1h2 = numpy.random.rand(self.hd2Nodes, self.hd1Nodes)/10-0.05
            self.wh2o  = numpy.random.rand(self.outNodes, self.hd2Nodes)/10-0.05
        
        self.lr = learningrate

        self.activation_function = lambda x: scipy.special.expit(x)
    
    # train the neural network
    def train(self, inputs_list, targets_list):
            # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
            # calculate signals into hidden1 layer
        hidden1_inputs = numpy.dot(self.wih1, inputs)
            # calculate the signals emerging from hidden layer
        hidden1_outputs = self.activation_function(hidden1_inputs)

            # calculate signals into hidden2 layer
        hidden2_inputs = numpy.dot(self.wh1h2, hidden1_outputs)
            # calculate the signals emerging from hidden layer
        hidden2_outputs = self.activation_function(hidden2_inputs)
        
            # calculate signals into final output layer
        final_inputs = numpy.dot(self.wh2o, hidden2_outputs)
            # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        
            # output layer error is the (target - actual)
        output_errors = (targets - final_outputs) * final_outputs * (1.0 - final_outputs)
        
            # hidden2 layer error is the output_errors, split by weights, recombined at hidden2 nodes
        hidden2_errors = numpy.dot(self.wh2o.T, output_errors) * hidden2_outputs * (1.0 - hidden2_outputs)

            # hidden1 layer error is the hidden2_errors, split by weights, recombined at hidden1 nodes
        hidden1_errors = numpy.dot(self.wh1h2.T, hidden2_errors) * hidden1_outputs * (1.0 - hidden1_outputs)
        

            # update the weights for the links between the hidden and output layers
        self.wh2o += self.lr * numpy.dot( output_errors, numpy.transpose(hidden2_outputs))
        
            # update the weights for the links between the hidden and output layers
        self.wh1h2 += self.lr * numpy.dot( hidden2_errors, numpy.transpose(hidden1_outputs))
        
            # update the weights for the links between the input and hidden layers
        self.wih1 += self.lr * numpy.dot( hidden1_errors, numpy.transpose(inputs))

    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden1_inputs = numpy.dot(self.wih1, inputs)
        # calculate the signals emerging from hidden layer
        hidden1_outputs = self.activation_function(hidden1_inputs)
        
        # calculate signals into hidden2 layer
        hidden2_inputs = numpy.dot(self.wh1h2, hidden1_outputs)
        # calculate the signals emerging from hidden layer
        hidden2_outputs = self.activation_function(hidden2_inputs)        
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.wh2o, hidden2_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs
    
    def saveWeight(self):
        numpy.save(WeightFile1 , self.wih1 )
        numpy.save(WeightFile2 , self.wh1h2)
        numpy.save(WeightFile3 , self.wh2o )


### дополнительные функции

In [ ]:
# inputs, targets
def get_inputs(x,y):
    return [y/height+0.1, x/width+0.1, 1]

def get_target(x,y):
    return [im[y,x,0]/256+0.001,im[y,x,1]/256+0.001,im[y,x,2]/256+0.001]

def train(epox, fileName):   
    fileName = fileName
    for h in range(0,epox):
        for n in range(0,400000):
            x = random.randint(0,width-1)
            y = random.randint(0,height-1)
            inputs = get_inputs(x,y)
            target = get_target(x,y)
            nn.train(inputs,target)
        generate_image()
        imageio.imwrite(folder+str(fileName)+'.png', imout[:, :, :])
        nn.saveWeight()
        fileName += 1
        
def generate_image():
    for x in range(0,width):
        for y in range(0,height):
            inputs = get_inputs(x,y)
            result = nn.query(inputs)
            imout[y,x,0] = result[0]*255
            imout[y,x,1] = result[1]*255
            imout[y,x,2] = result[2]*255
    

## обучение и просмотр результата
**epox** - количество прогонов (здесь 400000 рандомно-выбранных пиксель подаются на вход)    
В качестве второго параметра передаётся число, которое будет использоваться в генерации имени первой картинки, которые сохраняются в папке *progres*    
Как только результат удовлетворяет, то обучение можно прервать.    
Все веса сохраняются в соответствующей папке

In [ ]:
# создаём нейросеть
nn = neuralNetwork(3, neuron_count, neuron_count, 3, learning_rate, loadWeights)

epox = 1000
train(epox, 0)

# по окончании обучения, вывести картинку результата
generate_image()
matplotlib.pyplot.imshow(imout, interpolation='None')